In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from collections import defaultdict
from datetime import datetime
import h5py
import matplotlib.pyplot as plt
from neural_analysis.matIO import loadmat
import numpy as np
import os
import pandas as pd
from scipy.signal import butter, filtfilt, freqz
from scipy.stats import pearsonr
import shutil
import sys
import time
from tqdm.auto import tqdm

sys.path.append('../')
from ld_utils import compute_eigs, get_data_sigma_norms
from up_down import get_up_down
from utils import get_binary_stimuli, get_sample_interval, load, save, to_string

In [3]:
# session = 'Mary-Anesthesia-20160809-01'
session = 'Mary-Anesthesia-20170203-02'

## Load Neural Data and Split Into Segments

In [4]:
all_data_dir = f"/om/user/eisenaj/ChaoticConsciousness/data"
for (dirpath, dirnames, filenames) in os.walk(all_data_dir):
    if f"{session}.mat" in filenames:
        data_class = os.path.basename(dirpath)
        break

In [4]:
filename = os.path.join(all_data_dir, data_class, f'{session}.mat')
print("Loading data ...")
start = time.process_time()
electrode_info, lfp, lfp_schema, session_info, spike_times, unit_info = loadmat(filename, variables=['electrodeInfo', 'lfp', 'lfpSchema', 'sessionInfo', 'spikeTimes', 'unitInfo'], verbose=False)
spike_times = spike_times[0]
dt = lfp_schema['smpInterval'][0]
T = lfp.shape[0]

# f = h5py.File(filename, 'r')
# airPuff_binary, audio_binary = get_binary_stimuli(f)

print(f"Data loaded (took {time.process_time() - start:.2f} seconds)")

Loading data ...
Data loaded (took 202.74 seconds)


In [6]:
end_step = T - int(30/dt) # take off 30 seconds
data = lfp[:end_step]
for window in [2.5]:
    stride = 2.5
      
    data_dir = f"/om/user/eisenaj/ChaoticConsciousness/data/{data_class}/{session}_window_{window}_stride_{stride}"
    os.makedirs(data_dir, exist_ok=True)
    
    print(f"window = {window} s")
    num_windows = int(end_step/int(stride/dt))
    for i in tqdm(range(num_windows)):
        start = int(stride/dt)*i
        end = int(stride/dt)*i + int(window/dt)
        window_data = data[start:end]
        
        window_dict = {'data': window_data, 'start_ind': start, 'start_time': start*dt, 'end_ind': end, 'end_time': end*dt}
        save(window_dict, os.path.join(data_dir, f"window_{i}"))
        
#         print(f"start = {start}, end = {end}")

window = 2.5 s


  0%|          | 0/4141 [00:00<?, ?it/s]

## Compile Results and Save

In [3]:
folder_to_compile = "/om/user/eisenaj/ChaoticConsciousness/results/propofol/causality/causality_Mary-Anesthesia-20160809-01_window_2.5_stride_2.5"
assert os.path.exists(folder_to_compile)

In [7]:
def compile_folder(folder_to_compile):
    files = os.listdir(folder_to_compile)

    all_window_data = []
    for file in tqdm(files):
        window_data = load(os.path.join(folder_to_compile, file))
        all_window_data.append(window_data) 

    results_df = pd.DataFrame(all_window_data)

    results_df = results_df.sort_values('start_ind')
    results_df = results_df.reset_index(drop=True)

    shutil.rmtree(folder_to_compile)

    save(results_df, folder_to_compile)

In [8]:
results_dir = "/om/user/eisenaj/ChaoticConsciousness/results/propofolWakeup/VAR/VAR_Mary-Anesthesia-20170203-02_window_2.5_stride_2.5"
for folder in tqdm(os.listdir(results_dir)):
    compile_folder(os.path.join(results_dir, folder))

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/4141 [00:00<?, ?it/s]

  0%|          | 0/4141 [00:00<?, ?it/s]

  0%|          | 0/4141 [00:00<?, ?it/s]

  0%|          | 0/4141 [00:00<?, ?it/s]

  0%|          | 0/4141 [00:00<?, ?it/s]

  0%|          | 0/4141 [00:00<?, ?it/s]